In [105]:
import folium

import pandas as pd

import random

In [13]:
cities = pd.read_csv('5_infected_cities.csv')

In [15]:
cities['city'].values

array(['Baghdad', 'Chennai', 'Kuala Lumpur', 'Riyadh', 'Toronto'],
      dtype=object)

In [16]:
location_dict = {
    'Baghdad': [33.3152, 44.3661],
    'Chennai':[13.0827, 80.2707],
    'Kuala Lumpur': [3.1390, 101.6869],
    'Riyadh': [24.7136, 46.6753],
    'Toronto': [43.6532, -79.3832], 
}

In [23]:
def get_coords(row):
    return location_dict[row['city']]

In [24]:
cities[['lat', 'lon']] = cities.apply(get_coords, axis=1, result_type="expand")

In [40]:
cities['init_cases'] = cities['urban_pop'] * 0.01
cities['outbreak'] = (cities['urban_pop'] + cities['rest_of_country']) * 0.03

In [86]:
cities.loc[cities['city'] == 'Chennai', 'outbreak'] = 1506674

In [41]:
cities.head()

,city,country,country_pop,un_2018_pop,proper_pop,proper_area,metro_pop,metro_area,urban_pop,urban_area,rest_of_country,lat,lon,init_cases,outbreak
0,Baghdad,Iraq,40222493,"6,812,000",8126755,5200,NaN,NaN,5796000.0,693.0,3.442649e+07,33.3152,44.3661,57960.0,1206674.79
1,Chennai,India,1380004385,"10,456,000",6727000,426,NaN,NaN,11324000.0,1049.0,1.368680e+09,13.0827,80.2707,113240.0,41400131.55
2,Kuala Lumpur,Malaysia,32365999,"7,564,000",1768000,243,7200000.0,2793.0,8285000.0,2162.0,2.408100e+07,3.1390,101.6869,82850.0,970979.97
3,Riyadh,Saudi Arabia,34813871,"6,907,000",6694000,1913,NaN,NaN,6881000.0,1672.0,2.793287e+07,24.7136,46.6753,68810.0,1044416.13
4,Toronto,Canada,37742154,"6,082,000",2731571,630,5928040.0,5906.0,6871000.0,2334.0,3.087115e+07,43.6532,-79.3832,68710.0,1132264.62


In [91]:
cities['city'].values

array(['Baghdad', 'Chennai', 'Kuala Lumpur', 'Riyadh', 'Toronto'],
      dtype=object)

In [96]:
m_1 = folium.Map(
    location=[0, 0],
    #tiles='Stamen Toner',
    tiles='CartoDB positron',
    zoom_start=3
)

In [97]:
def add_marker(row):
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['init_cases'] / 10000,
        popup=row['city'],
        color='#FF6347',
        fill=True,
        fill_color='#FF6347',
    ).add_to(m_1)
    
    return

In [98]:
cities.apply(add_marker, axis=1)

0    None
1    None
2    None
3    None
4    None
dtype: object

In [99]:
m_1

In [106]:
m_1.save('initial_map.html')

In [100]:
m2 = folium.Map(
    location=[0, 0],
    tiles='CartoDB positron',
    zoom_start=3
)

def add_marker_2(row):
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['outbreak'] / 10000,
        popup=row['city'],
        color='#FF6347',
        fill=True,
        fill_color='#FF6347',
    ).add_to(m2)
    
    return

In [101]:
cities.apply(add_marker_2, axis=1)

m2

In [107]:
m2.save('country-level-outbreak.html')

In [30]:
world_centroids = pd.read_csv('country_centroids_az8.csv')
world_centroids.head()

,FID,the_geom,FID_,scalerank,featurecla,labelrank,sovereignt,sov_a3,adm0_dif,level,...,region_un,subregion,region_wb,name_len,long_len,abbrev_len,tiny,homepart,Longitude,Latitude
0,country_centroids_az8.1,POINT (-69.9826771125 12.5208803838),0,3,Admin-0 country,5,Netherlands,NL1,1,2,...,Americas,Caribbean,Latin America & Caribbean,5,5,5,4,-99,-69.982677,12.520880
1,country_centroids_az8.2,POINT (66.0047336558 33.8352307278),1,1,Admin-0 country,3,Afghanistan,AFG,0,2,...,Asia,Southern Asia,South Asia,11,11,4,-99,1,66.004734,33.835231
2,country_centroids_az8.3,POINT (17.5373676815 -12.2933605438),2,1,Admin-0 country,3,Angola,AGO,0,2,...,Africa,Middle Africa,Sub-Saharan Africa,6,6,4,-99,1,17.537368,-12.293361
3,country_centroids_az8.4,POINT (-63.0649892654 18.2239595023),3,1,Admin-0 country,6,United Kingdom,GB1,1,2,...,Americas,Caribbean,Latin America & Caribbean,8,8,4,-99,-99,-63.064989,18.223959
4,country_centroids_az8.5,POINT (20.0498339611 41.1424498947),4,1,Admin-0 country,6,Albania,ALB,0,2,...,Europe,Southern Europe,Europe & Central Asia,7,7,4,-99,1,20.049834,41.142450


In [198]:
world_cols = ['sovereignt', 'Longitude', 'Latitude', 'pop_est']
world = world_centroids.loc[:, world_cols]

In [199]:
final_cols = ['country', 'lat', 'lon', 'outbreak']

final_cities = cities.loc[:, final_cols]

final_cities['outbreak'] = final_cities['outbreak'] * 0.9

world.rename(columns = {
    'sovereignt': 'country', 
    'Latitude': 'lat',
    'Longitude': 'lon',
    'pop_est': 'outbreak',
}, inplace=True)

In [200]:
def rand_sick(pop):
    return pop * random.uniform(0.02, 0.05)

world['outbreak'] = world['outbreak'].apply(rand_sick)

In [221]:
m3 = folium.Map(
    location=[0, 0],
    tiles='CartoDB positron',
    zoom_start=3
)

def add_marker_3(row):
    
    folium.CircleMarker(
        location=[row['lat'], row['lon']],
        radius=row['outbreak'] / 100000,
        popup=row['country'],
        color='#FF6347',
        fill=True,
        fill_color='#FF6347',
    ).add_to(m3)
    
    return

In [222]:
world = world.sort_values('outbreak').drop_duplicates(subset='country', keep='last')

In [223]:
world.loc[world['country'] == 'United States of America', 'lat'] = 39.8283
world.loc[world['country'] == 'United States of America', 'lon'] = -98.5475

In [224]:
print(world.loc[world['country'] == 'Brazil', :])
print(world.loc[world['country'] == 'United States of America', :])

   country        lon        lat  outbreak
33  Brazil -53.097831 -10.787777  110000.0
                      country      lon      lat      outbreak
226  United States of America -98.5475  39.8283  1.416815e+07


In [225]:
world.loc[world['country'] == 'China', 'outbreak'] = 1200000
world.loc[world['country'] == 'Brazil', 'outbreak'] = 1100000

In [226]:
#outbreak_world = world.sample(frac = 0.9)
outbreak_world = world.copy()

In [227]:
outbreak_world.apply(add_marker_3, axis=1)

m3

In [228]:
m3.save('world_outbreak.html')